In [1]:
import json
import time
from datetime import datetime

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
from IPython.display import IFrame
from pyecharts import options as opts
from pyecharts.charts import Line, Map, Pie
from pypinyin import lazy_pinyin

plt.rcParams["font.sans-serif"] = ["FangSong"]
plt.rcParams["axes.unicode_minus"] = False

In [2]:
url = "https://view.inews.qq.com/g2/getOnsInfo?name=disease_h5&callback=&_=%d" % int(
    time.time() * 1000
)
data = json.loads(requests.get(url=url).json()["data"])
data.keys()
# data

dict_keys(['chinaTotal', 'chinaAdd', 'lastUpdateTime', 'areaTree', 'chinaDayList', 'chinaDayAddList', 'isShowAdd'])

# China Trend

In [3]:
data["chinaDayList"].sort(key=lambda x: x["date"])
# data["chinaDayList"]

In [4]:
date_list = []
confirmed_list = []
suspected_list = []
dead_list = []
cured_list = []
for item in data["chinaDayList"]:
    month, day = item["date"].split(".")
    date_list.append(
        datetime.date(
            datetime.strptime("2020-%s-%s" % (month, day), "%Y-%m-%d")
        ).isoformat()
    )
    confirmed_list.append(int(item["confirm"]))
    suspected_list.append(int(item["suspect"]))
    dead_list.append(int(item["dead"]))
    cured_list.append(int(item["heal"]))
confirm_ylist = [0] + confirmed_list[:-1]
confirm_addlist = [
    confirmed_list[i] - confirm_ylist[i] for i in range(len(confirmed_list))
]

In [5]:
file_name = "2019-nCoV_China_Trend"
line = Line()
line.add_xaxis(xaxis_data=date_list)
line.add_yaxis(
    series_name="Confirmed_Yesterday",
    stack="stack1",
    y_axis=confirm_ylist,
    areastyle_opts=opts.AreaStyleOpts(opacity=0.3),
    label_opts=opts.LabelOpts(is_show=False),
)
line.add_yaxis(
    series_name="Confirmed_Add",
    stack="stack1",
    y_axis=confirm_addlist,
    areastyle_opts=opts.AreaStyleOpts(opacity=0.5),
    label_opts=opts.LabelOpts(is_show=False),
)
line.add_yaxis(
    series_name="Confirmed_Today",
    stack="stack2",
    y_axis=confirmed_list,
    areastyle_opts=opts.AreaStyleOpts(opacity=0.0),
    #     label_opts=opts.LabelOpts(is_show=False),
)
line.set_global_opts(
    title_opts=opts.TitleOpts(title=file_name),
    #         datazoom_opts=[opts.DataZoomOpts(), opts.DataZoomOpts(type_='inside')],
    toolbox_opts=opts.ToolboxOpts(),
    tooltip_opts=opts.TooltipOpts(trigger="axis", axis_pointer_type="cross"),
    yaxis_opts=opts.AxisOpts(
        type_="value",
        axistick_opts=opts.AxisTickOpts(is_show=True),
        splitline_opts=opts.SplitLineOpts(is_show=True),
    ),
    xaxis_opts=opts.AxisOpts(type_="category", boundary_gap=False),
)
line.render(file_name + ".html")

IFrame(src="./" + file_name + ".html", width=1000, height=600)

'C:\\Users\\Eve\\Desktop\\2019-nCoV\\2019-nCoV_China_Trend.html'

# World Count

In [6]:
country = data["areaTree"]
country.sort(key=lambda x: x["total"]["confirm"], reverse=True)
# country

In [7]:
def list_generation(map_type):
    area_list = []
    confirmed_list = []
    suspected_list = []
    dead_list = []
    cured_list = []
    for item in map_type:
        area_list.append(item["name"])
        confirmed_list.append(int(item["total"]["confirm"]))
        suspected_list.append(int(item["total"]["suspect"]))
        dead_list.append(int(item["total"]["dead"]))
        cured_list.append(int(item["total"]["heal"]))
    return area_list, confirmed_list, suspected_list, dead_list, cured_list


area_clist, confirmed_clist, suspected_clist, dead_clist, cured_clist = list_generation(
    country
)

In [8]:
# def line_generation0(area_list, confirmed_list, suspected_list, dead_list, cured_list):
#     plt.figure(figsize=(20, 5))
#     plt.grid(color="r", linestyle="-.")
#     plt.plot(
#         area_list[1:],
#         confirmed_list[1:],
#         marker="o",
#         label="Confirmed_Count",
#         linestyle="-",
#     )
#     plt.plot(area_list[1:], suspected_list[1:], label="Suspected_Count")
#     plt.plot(area_list[1:], dead_list[1:], label="Dead_Count")
#     plt.plot(area_list[1:], cured_list[1:], label="Cured_Count")
#     plt.legend()


# line_generation0(area_clist, confirmed_clist, suspected_clist, dead_clist, cured_clist)

In [9]:
def line_generation(
    area_list, confirmed_list, suspected_list, dead_list, cured_list, map_type
) -> Line:
    if map_type == "world":
        file_name = "2019-nCoV_" + "World"
    elif map_type == "china" or map_type == "China" or map_type == "中国":
        file_name = "2019-nCoV_" + "China"
    else:
        province_pyname = "".join(lazy_pinyin(province_name))
        file_name = "2019-nCoV_" + province_pyname.title()
    c = (
        Line(init_opts=opts.InitOpts(width="1200px", height="500px"))
        .add_xaxis(area_list[1:])
        .add_yaxis(
            "Confirmed_Count",
            confirmed_list[1:],
            #             markline_opts=opts.MarkLineOpts(data=[opts.MarkLineItem(type_="max")]),
        )
        .add_yaxis(
            "Suspected_Count",
            suspected_list[1:],
            label_opts=opts.LabelOpts(is_show=False),
            #             markpoint_opts=opts.MarkPointOpts(data=[opts.MarkPointItem(type_="max"),]),
        )
        .add_yaxis(
            "Dead_Count",
            dead_list[1:],
            label_opts=opts.LabelOpts(is_show=False),
            #             markpoint_opts=opts.MarkPointOpts(data=[opts.MarkPointItem(type_="max"),]),
        )
        .add_yaxis(
            "Cured_Count",
            cured_list[1:],
            label_opts=opts.LabelOpts(is_show=False),
            #             markpoint_opts=opts.MarkPointOpts(data=[opts.MarkPointItem(type_="max"),]),
        )
        .set_series_opts(
            markpoint_opts=opts.MarkPointOpts(data=[opts.MarkPointItem(type_="max"),]),
        )
        .set_global_opts(
            title_opts=opts.TitleOpts(title=file_name + "_Line"),
            toolbox_opts=opts.ToolboxOpts(),
            tooltip_opts=opts.TooltipOpts(trigger="axis", axis_pointer_type="cross"),
            yaxis_opts=opts.AxisOpts(
                type_="value",
                axistick_opts=opts.AxisTickOpts(is_show=True),
                splitline_opts=opts.SplitLineOpts(is_show=True),
            ),
            xaxis_opts=opts.AxisOpts(
                type_="category",
                axislabel_opts=opts.LabelOpts(rotate=30, font_size=10),
                #                 axistick_opts=opts.AxisTickOpts(is_show=True),
                #                 boundary_gap=False,
            ),
        )
        .render(file_name + "_Line.html")
    )
    return IFrame(src="./" + file_name + "_Line.html", width=1300, height=600)


line_generation(
    area_clist, confirmed_clist, suspected_clist, dead_clist, cured_clist, "world"
)

In [10]:
with open("country.json", "r", encoding="utf-8") as f:
    country_data = dict(json.load(f))
country_ename = [x for i in area_clist for x, y in country_data.items() if i == y]
country_dict = dict(zip(country_ename, confirmed_clist))

In [11]:
def pie_generation(data_dict, map_type, skip=True):
    if map_type == "world":
        file_name = "2019-nCoV_" + "World"
    elif map_type == "china" or map_type == "China" or map_type == "中国":
        file_name = "2019-nCoV_" + "China"
    else:
        province_pyname = "".join(lazy_pinyin(province_name))
        file_name = "2019-nCoV_" + province_pyname.title()
    c = (
        Pie()
        .add(
            "",
            [list(z) for z in data_dict.items()][1:]
            if skip
            else [list(z) for z in data_dict.items()],
        )
        .set_series_opts(label_opts=opts.LabelOpts(formatter="{b}: {c}"))
        .set_global_opts(
            title_opts=opts.TitleOpts(title=file_name + "_Pie"),
            legend_opts=opts.LegendOpts(pos_left="90%", orient="vertical"),
        )
    )
    c.render(file_name + "_Pie.html")
    return IFrame(src="./" + file_name + "_Pie.html", width=1100, height=600)


pie_generation(country_dict, "world")

In [12]:
def map_generation(data_dict, map_type):
    map = Map()
    if map_type == "world":
        file_name = "2019-nCoV_" + "World"
        map.add(
            "Country_Count",
            data_pair=data_dict.items(),
            maptype=map_type,
            is_roam=True,
        )
        map.set_series_opts(label_opts=opts.LabelOpts(is_show=False))
    elif map_type == "china" or map_type == "China" or map_type == "中国":
        file_name = "2019-nCoV_" + "China"
        map.add(
            "Province_Count",
            data_pair=data_dict.items(),
            maptype="china",
            is_roam=True,
        )
    else:
        province_pyname = "".join(lazy_pinyin(province_name))
        file_name = "2019-nCoV_" + province_pyname.title()
        map.add(
            "City_Count", data_pair=data_dict.items(), maptype=map_type, is_roam=True,
        )
    map.set_global_opts(
        title_opts=opts.TitleOpts(title=file_name + "_Map"),
        toolbox_opts=opts.ToolboxOpts(),
        visualmap_opts=opts.VisualMapOpts(
            #             max_=30000
            is_piecewise=True,
            pieces=[
                {"max": 100000, "min": 10000, "label": ">=10000", "color": "#8A0808"},
                {"max": 9999, "min": 1000, "label": "1000-9999", "color": "#B40404"},
                {"max": 999, "min": 500, "label": "500-999", "color": "#DF0101"},
                {"max": 499, "min": 100, "label": "100-499", "color": "#eb6666"},
                {"max": 99, "min": 10, "label": "10-99", "color": "#F5A9A9"},
                {"max": 9, "min": 1, "label": "1-9", "color": "#fae5e5"},
                {"max": 0, "min": 0, "label": "0", "color": "#FFFFFF"},
            ],
        ),
    )
    map.render(file_name + "_Map.html")
    return IFrame(src="./" + file_name + "_Map.html", width=1000, height=600)


map_generation(country_dict, "world")

# China Count

In [13]:
country_name = "中国"
province = [
    item["children"] for item in data["areaTree"] if item["name"] == country_name
][0]

province.sort(key=lambda x: x["total"]["confirm"], reverse=True)
# province

In [14]:
area_plist, confirmed_plist, suspected_plist, dead_plist, cured_plist = list_generation(
    province
)
line_generation(
    area_plist, confirmed_plist, suspected_plist, dead_plist, cured_plist, country_name
)

In [15]:
province_dict = dict(zip(area_plist, confirmed_plist))

In [16]:
map_generation(province_dict, country_name)

In [17]:
pie_generation(province_dict, country_name, False)

# Province Count

In [18]:
province_name = "湖北"
city = [
    item["children"]
    for item in data["areaTree"][0]["children"]
    if item["name"] == province_name
][0]
city.sort(key=lambda x: x["total"]["confirm"], reverse=True)
# city

In [19]:
(
    area_ctlist,
    confirmed_ctlist,
    suspected_ctlist,
    dead_ctlist,
    cured_ctlist,
) = list_generation(city)

line_generation(
    area_ctlist,
    confirmed_ctlist,
    suspected_ctlist,
    dead_ctlist,
    cured_ctlist,
    province_name,
)

In [20]:
with open("city.json", "r", encoding="utf-8") as f:
    city_data = json.load(f)["provinces"]
city_flist = [
    item["cities"] for item in city_data if province_name in item["province"]
][0]
city_rlist = [
    city_flist[j]
    for i in range(len(area_ctlist))
    for j in range(len(city_flist))
    if area_ctlist[i][:2] in city_flist[j]
]
city_dict = dict(zip(city_rlist, confirmed_ctlist))

In [21]:
map_generation(city_dict, province_name)

In [22]:
pie_generation(city_dict, province_name, False)